In [43]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import json

# Load processed track IDs from a file
try:
    with open('processed_tracks.json', 'r') as file:
        processed_tracks = json.load(file)
except FileNotFoundError:
    processed_tracks = []

# Set up Spotify credentials
client_id = 'your_client_id'
client_secret = 'your_client_secret'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Set up Google Sheets credentials
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('credentials.json', scope)
client = gspread.authorize(credentials)
sheet = client.open('Spotify').sheet1

# Input the year to extract data for
year = input("Enter the year to extract data for: ")

# Input the type of songs (e.g., Spanish, pop, Hollywood, Punjabi)
song_type = input("Enter the type of songs you are looking for (e.g., Spanish, Pop, Bollywood, Punjabi): ")


# Search for tracks (limiting to 50 tracks per request)
offset = 0
limit = 50
all_tracks = []

try:
    while True:
        results = sp.search(q=f'year:{year} {song_type}', type='track', limit=limit, offset=offset)
        tracks = results['tracks']['items']
        if not tracks:
            break
        all_tracks.extend(tracks)
        offset += limit
except spotipy.SpotifyException as e:
    print("Error:", e)

# Extract information for each track
track_data = []
for track in all_tracks:
    if track['id'] not in processed_tracks:
        track_name = track['name']
        artists = ", ".join([artist['name'] for artist in track['artists']])
        album_name = track['album']['name']
        release_date = track['album']['release_date']
        track_id = track['id']
        # Add more fields as needed

        track_data.append([track_name, artists, album_name, release_date, track_id])
        processed_tracks.append(track_id)

        # # Check if the track or artist name contains Bollywood or Hindi keywords
        # if 'bollywood' in track_name.lower() or 'bollywood' in artists.lower() or 'bollywood' in album_name.lower():
        #     track_data.append([track_name, artists, album_name, release_date, track_id])
        #     processed_tracks.append(track_id)

# Save data to Google Sheet
sheet.insert_rows(track_data)

# Save updated processed track IDs to file
with open('processed_tracks.json', 'w') as file:
    json.dump(processed_tracks, file)

print("Data saved to Google Sheet.")


Enter the year to extract data for:  2023
Enter the type of songs you are looking for (e.g., Spanish, Pop, Bollywood, Punjabi):  Punjabi


HTTP Error for GET to https://api.spotify.com/v1/search with Params: {'q': 'year:2023 Punjabi', 'limit': 50, 'offset': 1000, 'type': 'track', 'market': None} returned 400 due to Bad request.


Error: http status: 400, code:-1 - https://api.spotify.com/v1/search?q=year%3A2023+Punjabi&limit=50&offset=1000&type=track:
 Bad request., reason: None
Data saved to Google Sheet.
